In [10]:
from __future__ import print_function, division

import os

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
#from keras.layers.convolutional import UpSampling2D, Conv2D,Conv2DTranspose
from keras.layers.convolutional import Conv2D,Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, Adam
from keras.initializers import RandomNormal
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
import matplotlib.pyplot as plt
import sys
import numpy as np
from keras.utils import multi_gpu_model
import zipfile
import glob
import shutil as sh

Using TensorFlow backend.


In [5]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout,BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers.

SyntaxError: invalid syntax (<ipython-input-5-53460c15aa48>, line 4)

In [ ]:
def rescaleimage(image):
    image=(image-127.5)/127.5
    return image
def reverseimage(image):
    image=(image/2)+0.5
    return image

In [12]:
'''putting everything in one folder for augmentaition'''
allinone = '/home/cc/Data/Images'

files = glob.glob('/home/cc/Data/Allimages/Images/*.bmp', recursive=True)
files2 = glob.glob('/home/cc/Data/Allimages/Images/*.png', recursive=True)
filesall = files+files2

In [13]:
len(filesall)

227905

In [ ]:
p = Augmentor.Pipeline(allinone)

p.rotate90(probability=0.3)
p.rotate270(probability=0.3)
p.flip_left_right(probability=0.8)
p.flip_top_bottom(probability=0.3)
p.crop_random(probability=0.5, percentage_area=0.7)
p.resize(probability=1.0, width=512, height=512)


In [ ]:
os.remove('/home/cc/Data/Allimages/Images/200318113106_NM1_20150727113416_06_enhanced.png')

In [ ]:
img_rows = 512
img_cols = 512
channels = 3 # I am using channel last models
img_shape = (img_rows, img_cols, channels)
latent_dim = 512
Ngpus=1
batch_size=64*Ngpus

Imsize=(img_cols,img_cols)

datagen=ImageDataGenerator()
                          
image_generator=datagen.flow_from_directory(
    allinone,
    color_mode='rgb',
    batch_size= batch_size,
    target_size=(img_rows,img_cols),
    class_mode='input',
    shuffle=True
    )

In [ ]:
img_rows = 512
img_cols = 512
channels = 3 # I am using channel last models
img_shape = (img_rows, img_cols, channels)
latent_dim = 512
Ngpus=2
batch_size=64*Ngpus

Imsize=(img_cols,img_cols) # inception requirement
datagen=ImageDataGenerator(preprocessing_function=p.keras_preprocess_func())
                          
image_generator=datagen.flow_from_directory(
    allinone,
    color_mode='rgb',
    batch_size= batch_size,
    target_size=(img_rows,img_cols),
    class_mode='input',
    shuffle=True
    )

In [ ]:
initialize=RandomNormal(mean=0.0, stddev=0.02, seed=None)
from keras.regularizers import l2
'''
G_ENC
'''
CAE = Sequential()
CAE.add(Conv2D(16, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, input_shape=(512,512,3), padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(32, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(64, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(128, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(256, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(512, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(1024, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Flatten())
CAE.add(Dense(512))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

'''
G-DEC
'''

CAE.add(Dense(1024 * 4 * 4))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Reshape((4, 4, 1024)))
CAE.add(Conv2DTranspose(512, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))        
CAE.add(Conv2DTranspose(256, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(128, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))        
CAE.add(Conv2DTranspose(64, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(32, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(16, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(3, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('tanh'))

optimizer=Adam(lr=0.0001, beta_1=0.5, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=True)
CAE.compile(loss='mse', optimizer=optimizer)

In [ ]:
CAE.fit_generator(image_generator, epochs=100)

In [ ]:
from keras.models import load_model
#CAE = load_model('/home/cc/Data/Model_CAE_v3_final3')
#parallel_model = multi_gpu_model(CAE, gpus=Ngpus, cpu_relocation=False)
#optimizer=Adam(lr=0.00001, beta_1=0.5, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
#parallel_model.compile(loss='mse', optimizer=optimizer)

In [ ]:
from keras.callbacks import TensorBoard, ModelCheckpoint
TS = TensorBoard(log_dir='/home/cc/Data/Tests/logs', histogram_freq=0, batch_size=batch_size)
MC = ModelCheckpoint(os.path.join('/home/cc/Data/Tests','MC'), 
                        monitor='loss', verbose=0, save_best_only=True, 
                        save_weights_only=True, mode='auto', period=1)

parallel_model.fit_generator(g, steps_per_epoch=len(p.augmentor_images)/batch_size, epochs=100, 
                             verbose=1, max_queue_size=30, workers=6, use_multiprocessing=True, callbacks=[TS, MC])

In [ ]:
CAE.save('/home/cc/Data/Model_CAE')

In [ ]:
x1 = image_generator.next()[0]
x = CAE.predict(x1)

In [ ]:
img1 = []
img2 = []
N = [0,1,2, 3, 4, 5,]
for n in N:
    img1.append(x1[n,:,:,:])
    img2.append(x[n,:,:,:])

In [ ]:
for n in N:
    img1[n] = img1[n]/2
    img1[n] = img1[n]+0.5
    img2[n] = img2[n]/2
    img2[n] = img2[n]+0.5

In [ ]:
import matplotlib.pyplot as plt
i=1
plt.figure(figsize=(10, 10))
for n in N:
    plt.subplot(6,2,i)
    plt.imshow(img1[n])
    i= i+1
    plt.subplot(6,2,i)
    plt.imshow(img2[n])
    i=i+1

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img2[0])